# 3.2 在模型開發階段中，進行實驗性的訓練

使用手寫數字資料集訓練卷積神經網路，並以MLflow追蹤實驗結果

這份範例的目的是讓使用者了解[MLflow](https://mlflow.org/)的實驗追蹤、模型版本控制、部署等功能，我們將實際操作以下步驟：

1. 在[PyTorch](https://pytorch.org/)框架下使用[MNIST](http://yann.lecun.com/exdb/mnist/)資料集來訓練簡單的卷積神經網路模型，以不同超參數進行數次實驗
2. 以MLflow紀錄實驗結果
3. 在MLflow的UI進行模型註冊（Register model）
4. 以MLflow的API讀取特定版本的模型，並用來推論

這份Jupyter Notebook著重在使用[MLflow Python API](https://mlflow.org/docs/latest/python_api/index.html)來完成以上的步驟，除了紀錄實驗結果要寫在訓練的程式碼，其他步驟都能使用UI來操作。更多資訊可參考MLflow的[官方教學](https://mlflow.org/docs/latest/tutorials-and-examples/index.html)。

## 需要的套件

In [ ]:
import os
import zipfile
from datetime import datetime
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
from torchvision import transforms, models
from torchinfo import summary
import numpy as np
import mlflow
from dotenv import load_dotenv

## 下載資料集、指定PyTorch的運算裝置

In [ ]:
# 若機器配備支援CUDA的GPU請執行這邊
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 若機器運行macOS且配備M系列晶片或特定AMD顯示卡請執行這邊
# 相關說明請見https://developer.apple.com/metal/pytorch/
# DEVICE = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

print(f'Training device: {DEVICE}')

## 設定MLflow的相關環境變數、建立實驗

這些MLflow相關的環境變數很重要，如果有些項目沒正確設定，可能導致MLflow無法存取伺服器上的資源

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['MLFLOW_S3_ENDPOINT_URL'] = os.getenv('MLFLOW_S3_ENDPOINT_URL')
os.environ['LOGNAME'] = os.getenv('NB_USER')  # 設定要紀錄在實驗的使用者名稱

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
print(f'MLflow tracking URI: {mlflow.get_tracking_uri()}')

experiment_name = 'MNIST'

# 建立實驗
existing_exp = mlflow.get_experiment_by_name(experiment_name)
if not existing_exp:
    mlflow.create_experiment(experiment_name, "s3://mlflow/")
    
mlflow.set_experiment(experiment_name)

### 資料前處理

1. 前處理
2. 切割訓練、驗證集
3. 建立訓練、驗證集的[DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

In [ ]:
# 指定驗證集佔比
val_size = 0.2
# 是否對資料進行洗牌
shuffle_data = True
# 隨機種子
random_seed = 1
# Batch size
batch_size = 256

# 1. 前處理
transform_list_aug = transforms.Compose(
    [
        transforms.Grayscale(),  # MNIST需要將圖片轉換為灰階，否則預設是3通道圖片
        transforms.Resize([28, 28]),
        transforms.ToTensor(),
    ]
)

train_data = torchvision.datasets.ImageFolder(
    root='./data/MNIST/train/',
    transform=transform_list_aug
)

# 2. 切割訓練、驗證集
indices = list(range(len(train_data)))
split_point = int(np.floor(val_size * len(train_data)))

if shuffle_data:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split_point:], indices[:split_point]
print(f'Training set size: {len(train_indices)}, validation set size: {len(val_indices)}')

# 3. 建立訓練與驗證集的DataLoader
train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_indices),
    num_workers=2
)
val_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_indices),
    num_workers=2
)

### 建立模型

In [ ]:
class Net(nn.Module):
    """
    針對手寫數字辨識建立簡單的CNN模型
    """
    def __init__(self):
        super(Net, self).__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding='same'
            ),
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2
            )
        )
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=4,
                kernel_size=5,
                stride=1,
                padding='same'
            ),
            nn.ReLU(),
        )
        self.flatten = nn.Flatten()
        self.classify = nn.Linear(
            in_features=4 * 14 * 14,
            out_features=10
        )

    def forward(self, x):
        x = self.stem(x)
        x = self.conv1(x)
        x = self.flatten(x)
        output = self.classify(x)

        return output

In [ ]:
net = Net()

# 使用者自訂的epoch數量
n_epochs = 24
i_iter = 0

# 指定最佳化器（Optimizer）、損失函數（Loss function）
optimizer = torch.optim.AdamW(net.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

# 存放每個epoch的loss, acc
train_losses = []
valid_losses = []
train_accs = []
valid_accs = []

net.to(DEVICE)

for epoch in range(n_epochs):
    # 存放一個epoch內每一個batch的loss, acc
    b_train_loss = []
    b_valid_loss = []
    b_train_acc = []
    b_valid_acc = []
    
    # 訓練
    net.train()
    for idx, (imgs, train_true_labels) in enumerate(train_loader):
        n_correct_train = 0

        # 訓練資料、標籤移動到DEVICE的記憶體
        imgs = imgs.float().to(DEVICE)
        train_true_labels = train_true_labels.to(DEVICE)
        
        # 最佳化器梯度歸零
        optimizer.zero_grad()
        
        # 前向傳遞
        train_outputs = net(imgs)
        
        # 計算Loss、反向傳遞
        train_loss = loss_fn(train_outputs, train_true_labels)
        train_loss.backward()
        optimizer.step()

        # 計算訓練集的預測標籤、計算正確率（Accuracy）
        train_outputs_label = torch.argmax(train_outputs, 1)
        n_correct_train = len(torch.where(train_outputs_label == train_true_labels)[0]) / len(train_true_labels)

        # 紀錄這個epoch的訓練集loss, acc
        b_train_loss.append(train_loss.item())
        b_train_acc.append(n_correct_train)
    
    # 驗證
    n_correct_val = 0
    n_val_data = 0
    
    net.eval()
    with torch.no_grad():
        for _idx, (imgs, val_true_labels) in enumerate(val_loader):
            # 訓練資料、標籤移動到DEVICE的記憶體
            imgs = imgs.float().to(DEVICE)
            val_true_labels = val_true_labels.to(DEVICE)
            
            # 前向傳遞
            val_outputs = net(imgs)
            
            # 計算Loss
            val_loss = loss_fn(val_outputs, val_true_labels).item()

            # 計算驗證集的預測標籤、計算正確率（Accuracy）
            val_outputs_label = torch.argmax(val_outputs, 1)
            n_correct_val = len(torch.where(val_outputs_label == val_true_labels)[0])/len(val_true_labels)

            # 紀錄這個epoch的驗證集loss, acc
            b_valid_loss.append(val_loss)
            b_valid_acc.append(n_correct_val)

    # 計算這個epoch裡每一次迭代的loss, acc平均值
    ep_train_loss = np.mean(b_train_loss)
    ep_vaild_loss = np.mean(b_valid_loss)
    ep_train_acc = np.mean(b_train_acc)
    ep_valid_acc = np.mean(b_valid_acc)

    # 紀錄這個epoch裡每一次迭代的loss, acc平均值
    train_losses.append(ep_train_loss)
    valid_losses.append(ep_vaild_loss)
    train_accs.append(ep_train_acc)
    valid_accs.append(ep_valid_acc)
    
    # 印出每一次epoch的loss, acc
    print(
        f'{epoch + 1:2d}/{n_epochs:2d} {idx + 1:3d}/{len(train_loader):3d}, \
        train loss: {ep_train_loss:8.5f}, \
        train acc: {ep_train_acc:7.5f}, \
        val loss: {ep_vaild_loss:8.5f}, \
        val acc: {ep_valid_acc:7.5f}           '
    )
    i_iter += 1

# 將模型存檔
torch.save(net, f'./mnist.pt')

### 紀錄模型的訓練結果

MLflow[常用的實驗追蹤功能](https://mlflow.org/docs/latest/tracking.html#logging-functions)：
  * `mlflow.set_tracking_uri()`：指定MLflow追蹤的結果的儲存位址。
  * `mlflow.set_experiment()`：設定實驗名稱，建議可使用專案來命名。
  * `mlflow.start_run(run_name='Run_01')`：開始記錄實驗，`run_name='Run_01'`代表這一次執行的名稱為Run_01。系統預設是隨機產生出單字來代表，建議可自訂命名格式，比較方便管理。
  * `mlflow.log_params({'Optimizer': type(optimizer).__name__})`：紀錄實驗參數。可以用鍵值對的方式紀錄模型的學習率、資料版本等等。
  * `mlflow.log_metric('metric', value, step=i_iter)`：紀錄指標。例如紀錄每次迭代後的準確率、誤差，`'metric'`則是指標的名稱、`value`是模型訓練的訓練誤差、`step=i_iter`則紀錄了這次更新是第幾步。
  * `mlflow.log_model(net, artifact_path='Model')`：儲存模型。其中`net`為要儲存的模型，`artifact_path='Model'`則指定要將模型儲存在MLflow每一次執行的相對路徑，例如前述用法則會將模型儲存在`Model/`路徑底下，MLflow對常見的開發框架如scikit-learn或PyTorch、TensorFlow都有支援。

In [ ]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H-%M-%S")

net = Net()


with mlflow.start_run(run_name=f'Run_{dt_string}'):
    run_id = mlflow.active_run().info.run_id
    mlflow.set_experiment_tag('developer', 'AIF')
    mlflow.set_tags({
        'Framework': 'PyTorch',
        'Training device': DEVICE.type,
        'Phase': 'Experiment'
    })
    
    # 紀錄實驗的參數
    mlflow.log_params({
        'Model': net.__class__.__name__,
        'Number of epochs': n_epochs,
        'Optimizer': type(optimizer).__name__,
        'Initial learning rate': optimizer.param_groups[0]['lr'],
    })
    
    # 紀錄訓練過程的loss, acc
    for i in range(len(train_losses)):
        mlflow.log_metrics(
            {
                "Training loss": train_losses[i],
                "Validation loss": valid_losses[i],
                "Training accuracy":train_accs[i],
                "Validation accuracy":valid_accs[i]
            }, 
            step=i
        )
    
    # 儲存模型
    mlflow.pytorch.log_model(
        net, 
        artifact_path='Model',
        # registered_model_name='MNIST'
    )  # 如果再加上registered_model_name='MNIST'則會自動建立'MNIST'模型並且將目前的模型註冊為第1版

## 註冊模型

這部分除了可以[用MLflow的UI操作](https://mlflow.org/docs/latest/model-registry.html)，也能以程式碼來執行，如以下步驟：

## 建立一個新的模型

使用[mlflow.register_model](https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.register_model)註冊新模型，名稱為使用者自訂。

> 如果這個名字的模型已經存在，會印出Registered model 'DogCat' already exists. Creating a new version of this model...

In [ ]:
# 要註冊的名稱
model_name = 'MNIST'
model_uri = f'runs:/{run_id}/Model'

# 進行模型註冊
model_details = mlflow.register_model(
    model_uri=model_uri, 
    name=model_name
)

## 增加模型描述

要對MLflow伺服器的實驗結果或模型進行修改的話，通常會建立[mlflow.tracking.MlflowClient()](https://mlflow.org/docs/latest/python_api/mlflow.client.html)來達成。例如當我們要對已經建立好的模型增加描述，可以透過[client.update_registered_model()](https://mlflow.org/docs/latest/python_api/mlflow.client.html#mlflow.client.MlflowClient.update_registered_model)，而`mlflow.tracking.MlflowClient()`也還有非常多功能，未來若有需求也可自行閱讀。

In [ ]:
# 建立MLflow client來存取MLflow伺服器上的資料
client = mlflow.tracking.MlflowClient()

# 更新註冊的模型，增加一些描述
client.update_registered_model(
    name=model_details.name,
    description='Handwritten digits classification.'
)

## 切換模型狀態

這邊同樣用到剛才建立的`client`，透過它我們能控制模型的狀態，例如要進行部署或是封存等等

In [ ]:
# 以名稱來搜尋模型
model_version_infos = client.search_model_versions(f"name = '{model_name}'")
new_model_version = max([model_version_info.version for model_version_info in model_version_infos])

print(f'New model version: {new_model_version}')

In [ ]:
client.transition_model_version_stage(
  name=model_name,
  version=new_model_version,
  stage='Production',
)

## 讀取最新版本的模型並用來推論

In [ ]:
logged_model_info = client.get_latest_versions(name='MNIST', stages=["Production"])[0]  # 搜尋目前為Production的模型
print(f'Model version: {logged_model_info.version}')

logged_model_run_id = logged_model_info.run_id  # 模型的run_id
logged_model_path = f'runs:/{logged_model_run_id}/Model'  # 格式'runs:/<model_run_id>/Model'為MLflow固定的寫法，可透過特定run_id來讀取資料

# 讀取PyTorch模型
loaded_model = mlflow.pytorch.load_model(
    model_uri=logged_model_path,
    map_location=DEVICE
)
loaded_model.eval()

print(f'Logged model: \n{loaded_model}')

In [ ]:
# 列出模型架構
summary(loaded_model, input_size=(1, 1, 28, 28))

## 對範例圖片進行推論

In [ ]:
img = Image.open(f'./data/MNIST/train/3/10097.jpg')
img

In [ ]:
x = transform_list_aug(img)
x = x.unsqueeze(0).to(DEVICE)

print(loaded_model(x).detach())

## 補充：刪除實驗

如果我們想刪除實驗，除了透過UI，也可以透過Python API來完成。

`mlflow.delete_experiment(experiment_id)`：`experiment_id`是每個實驗的代碼，可以在一開始建立實驗時看到。

也能透過指令列介面來完成，詳見[MLflow Command-Line Interface的官方文件](https://mlflow.org/docs/0.6.0/cli.html)。

In [ ]:
# mlflow.delete_experiment(3)

也能透過指令列介面來完成，例如要刪除實驗：`mlflow experiments restore --experiment-id 3`，`--experiment-id 3`的3是要刪除的實驗代碼。

In [ ]:
# !mlflow experiments restore --experiment-id 3